# 50 Shades of face

In this notebook we aim to evalute the impact of different face crops in FR baselines.
For that we are relying on the MOBIO dataset, which is not ideal, but it's short enough to run a bunch of experiments.

In [ ]:
# Fetching resources
import bob.bio.base
from bob.bio.base.pipelines import execute_pipeline_simple
from bob.bio.base.algorithm import Distance
from bob.bio.base.pipelines import PipelineSimple
from bob.pipelines import wrap
from bob.bio.face.preprocessor import FaceCrop
from bob.bio.face.database import MobioDatabase
from clapp.rc import UserDefaults
import os
import scipy.spatial
import bob.measure

rc = UserDefaults("bobrc.toml")
dask_client = None

###
image_size = 112

# eyes position in the vertical axis
# final position will be image_size/height_denominators
height_denominators = [4.5,4,3.5,3,2.8]

# Eyes distance to be explored
eyes_distances = [30, 35, 40, 42,45,48]



output_path = "./50-shades"


######## CHANGE YOUR FEATURE EXTRACTOR HERE
from bob.bio.face.embeddings.mxnet import ArcFaceInsightFace_LResNet100
extractor_transformer = wrap(["sample"],ArcFaceInsightFace_LResNet100())

### CHANGE YOUR MATCHER HERE
algorithm = Distance(distance_function = scipy.spatial.distance.cosine,is_distance_function = True)


##### CHANGE YOUR DATABASE HERE
database = MobioDatabase(protocol="mobile0-male")
sample = database.references()[0][0]
import matplotlib.pyplot as plt
import bob.io.image




## Setting up the grid

If you want to run this on the cluster, don't forget to `SETSHELL grid` before running the cell below.


In [ ]:
# Starting dask client


from dask.distributed import Client
from bob.pipelines.distributed.sge import SGEMultipleQueuesCluster

cluster = SGEMultipleQueuesCluster(min_jobs=1)
dask_client = Client(cluster)


## Running different face crops

Here we are varying the `eyes_distances` and the ration `fig_size/height_denominators`,
generating the transformers and plotting the outcome

In [ ]:
%matplotlib widget

annotation_type = "eyes-center"
fixed_positions = None


def get_cropers(final_size = 112,
                height_denominators = [5,4,3,2],
                eyes_distances = [30, 35, 40, 42,45]):
    
    left_eye_offset = 1.49

    transformers = []
    for e in eyes_distances:
        for h in height_denominators:

            right_eye_offset = (final_size*left_eye_offset)/(final_size-e*left_eye_offset)
        
            RIGHT_EYE_POS = (final_size / h, final_size/right_eye_offset)
            LEFT_EYE_POS = (final_size / h, final_size/left_eye_offset)


            #RIGHT_EYE_POS = (final_size / 3.44, final_size / 3.02)
            #LEFT_EYE_POS = (final_size / 3.44, final_size / 1.49)
            cropped_positions = {
                "leye": LEFT_EYE_POS,
                "reye": RIGHT_EYE_POS,
            }
            #print(cropped_positions)

            preprocessor_transformer = FaceCrop(cropped_image_size=(112,112),
                                                cropped_positions=cropped_positions,
                                                color_channel='rgb',
                                                fixed_positions=fixed_positions)

            transform_extra_arguments = (None if (cropped_positions is None or fixed_positions is not None) else (("annotations", "annotations"),))

            preprocessor_transformer = wrap(["sample"], preprocessor_transformer,transform_extra_arguments=transform_extra_arguments)
            
            transformers.append(preprocessor_transformer)

    return transformers

def plot_faces(transformers, database, subplot_shape, fnmrs=None):

    fig, axis = plt.subplots(subplot_shape[0], subplot_shape[1])
    offset = 0
    for ax_h in axis:
        for ax_w in ax_h:
    
            # Picking the first sample
            sample = database.references()[0][0]
            preprocessor_transformer = transformers[offset]

            cropped = preprocessor_transformer.transform([sample])[0]
            cropped = bob.io.image.to_matplotlib(cropped.data).astype("uint8")

            ax_w.imshow(cropped)
            reye_y = round(preprocessor_transformer.estimator.cropped_positions["reye"][0],2)
            reye_x = round(preprocessor_transformer.estimator.cropped_positions["reye"][1],2)
            leye_y = round(preprocessor_transformer.estimator.cropped_positions["leye"][0],2)
            leye_x = round(preprocessor_transformer.estimator.cropped_positions["leye"][1],2)
            
            if fnmrs is None:
                title = f"({reye_y},{reye_x}) - ({leye_y},{leye_x})"
            else:
                title = f"({reye_y},{reye_x}) - ({leye_y},{leye_x}) = {fnmrs[offset]}"

            ax_w.set_title(f"{title}", fontsize=5)
            ax_w.axis('off')
            
            offset +=1
    

subplot_shape = (int((len(eyes_distances)*len(height_denominators))/len(height_denominators)),len(height_denominators))

transformers = get_cropers(final_size=image_size,
                           height_denominators=height_denominators,
                           eyes_distances=eyes_distances)

plot_faces(transformers, database, subplot_shape)


## Run PipelineSimple

Here we are running PipelineSimple several times and collecting the `1-FNMR@FMR=0.001` and plotting.

In [ ]:
"""
execute_pipeline_simple(
    pipeline,
    database,
    dask_client,
    groups,
    output,
    write_metadata_scores,
    checkpoint,
    dask_partition_size,
    dask_n_workers,
)
"""
from sklearn.pipeline import make_pipeline
write_metadata_scores = False
checkpoint = False
dask_partition_size = None
dask_n_workers = 15

### Preparing the pipeline

fnmrs = []
for t in transformers:

    # Chain the Transformers together
    transformer = make_pipeline(t, extractor_transformer)

    # Assemble the Biometric pipeline and execute
    pipeline = PipelineSimple(transformer, algorithm)

    execute_pipeline_simple(
        pipeline,
        database,
        dask_client,
        ["dev"],
        output_path,
        write_metadata_scores,
        checkpoint,
        dask_partition_size,
        dask_n_workers,
        score_all_vs_all=True
    )
    
    scores_dev = os.path.join(output_path, "scores-dev")
    
    # Picking FNMR@FAR=0.001
    neg, pos = bob.bio.base.score.load.split_four_column(scores_dev)
    far_thres = bob.measure.far_threshold(neg, pos, 0.001)
    fpr,fnr = bob.measure.fprfnr(neg, pos, far_thres)
    fnmr_1 = round(1-fnr,2)
    fnmrs.append(fnmr_1)
    
    
plot_faces(transformers, database, subplot_shape, fnmrs)        


In [ ]:
# Shutting down client
dask_client.shutdown()